In [ ]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd
import re
from datetime import datetime
import time

#set topages
page_num = 20
page_start = 0

AI = 'AI'
CV = 'computer+vision'
NLP = 'natural+language+processing'
DS = 'data+science'
ML = 'machine+learning'
domain = ML

NY = 'New+York'
SF = 'San+Francisco'
SE = 'Seattle'
region = SE

#set start page
page_url = 'https://www.indeed.com/jobs?q=' + domain + '&l=' + region
print(page_url)
# headers ={'User‐Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKi t/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}

#data
companyName = []
companyLocation = []
ratingNumber = []
salary = []
jobTitle = []
url_add = []
url_text = []
date = []
extract_date = []

while(True):
    page_num = page_num -1
    page_start = page_start+10
    if(page_num<0):
        print('all page done...')
        break
    else:
        print('left page...',page_num+1)

        page = requests.get(page_url)    
        if (page.status_code==200):   
                soup = BeautifulSoup(page.content, 'html.parser') 
                time.sleep(1)
                
                #next page
                page_url = page_url + '&start=' + str(page_start)
#                 temp=soup.find('ul',class_='pagination-list')
#                 temp=temp.find_all('a',href=True)
#                 next_url = 'https://www.indeed.com'+str(temp[0]['href'])
#                 page_url = next_url

                ##find main zone
                temp=soup.find('div',id='mosaic-zone-jobcards')
                temp=soup.find_all('a',attrs={"id":re.compile(r"sj_(\s\w+)?|job_(\s\w+)?")})

                for i in temp:
                    #companyName
                    result=i.find('span',class_='companyName')
                    if (result==None):
                        companyName.append('None')
                    else:
                        companyName.append(result.text)

                    #companyLocation
                    result=i.find('div',class_='companyLocation')
                    if (result==None):
                        companyName.append('None')
                    else:
                        companyLocation.append(result.text)

                    #ratingNumber
                    result=i.find('span',class_='ratingNumber')
                    if (result==None):
                        ratingNumber.append('None')
                    else:
                        ratingNumber.append(result.text)

                    #salary
                    result=i.find('div',class_='metadata salary-snippet-container')
                    if (result==None):
                        salary.append('None')
                    else:
                        salary.append(result.text)

                    #job title（remove new）
                    result=i.find('h2',attrs={"class":re.compile(r"jobTitle(\s\w+)?")})
                    if (result==None):
                        jobTitle.append('None')
                    else:
                        jobTitle.append(result.text)
                        
                    #date
                    result=i.find('span',class_='date')
                    if (result==None):
                        date.append('None')
                    else:
                        date.append(result.text)
                    
                    #extract date
                    extract_date.append(datetime.now())

                    #url
                    result = i['href']
                    if (result==None):
                        url_add.append('None')
                    else:
                        url_add.append('https://www.indeed.com'+str(result))

                    #url_content
                    page2 = requests.get('https://www.indeed.com'+str(result))    
                    if (page2.status_code==200):   
                            soup2 = BeautifulSoup(page2.content, 'html.parser')
                            time.sleep(1)
                            result=soup2.find('div',id='jobDescriptionText')

                            if (result==None):
                                url_text.append('None')
                            else:
                                url_text.append(result.text)


# build dataframe
data1 = {
"JobTitle":jobTitle,
"CompanyName":companyName,
"CompanyLocation":companyLocation,
"CompanyRating":ratingNumber,
"Salary":salary,
"Date":date,
"Extract_Date":extract_date,
"Content_url":url_add,
"Content_text":url_text
}
df1 = pd.DataFrame(data1)

df1.to_csv(domain+'_'+region+'.csv',index=False,header=True)